# Feature Importance Using Random Forest

### Overview

Feature Selection using Random Forest algorithm

### Builds on
None

### Run time
approx. 20 minutes

### Notes


## Step 1 - Load Data

In [1]:
import os
import urllib.request

data_location = "house-sales-full.csv"
data_url = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
#data_url = 'https://raw.githubusercontent.com/elephantscale/datasets/master/house-prices/house-sales-full.csv'

if not os.path.exists (data_location):
    data_location = os.path.basename(data_location)
    if not os.path.exists(data_location):
        print("Downloading : ", data_url)
        urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)


data_location: house-sales-full.csv


In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

house_prices = pd.read_csv(data_location)
house_prices.sample(10)

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
4534,4535,2/11/13,200000,1563101080,Single Family,2/1/13,347000,0.80,"250,836.00",1,...,1.50,2,6,1959,0,0,258000,5000,98116,False
6104,6105,9/27/06,315000,1982201521,Single Family,9/1/06,414800,0.95,"330,492.00",1,...,1.00,2,5,1910,0,0,161000,180000,98107,False
21328,21329,10/11/10,790000,7851980210,Single Family,10/1/10,334700,0.77,"1,027,212.00",1,...,3.25,4,10,2000,0,0,305000,619000,98065,False
718,719,12/11/07,1700000,269000090,Single Family,12/1/07,425200,0.98,"1,739,981.00",1,...,3.50,4,10,2006,0,0,333000,856000,98199,False
1260,1261,4/10/08,454000,475000080,Single Family,4/1/08,411700,0.95,"479,915.00",1,...,1.50,2,7,1904,0,0,268000,239000,98107,False
14839,14840,1/11/08,257500,4365700055,Single Family,1/1/08,421900,0.97,"265,617.00",1,...,1.00,1,6,1925,0,0,108000,94000,98106,False
17826,17827,2/19/09,401000,6396000170,Single Family,2/1/09,374400,0.86,"466,120.00",1,...,2.25,3,7,1993,0,0,217000,165000,98034,False
15302,15303,8/4/06,345000,5100403754,Single Family,8/1/06,411100,0.94,"365,225.00",1,...,1.00,3,6,1922,0,0,233000,156000,98115,False
20988,20989,9/28/06,345450,7697910040,Single Family,9/1/06,414800,0.95,"362,439.00",1,...,2.00,3,7,1990,0,0,128000,124000,98030,False
3673,3674,4/9/07,359000,1328330850,Single Family,4/1/07,430900,0.99,"362,583.00",1,...,1.75,3,8,1981,0,0,120000,207000,98058,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [3]:
house_prices.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [4]:
house_prices.corr()

<ipython-input-4-61669d780222>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  house_prices.corr()


,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data
Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [5]:
input_columns= ['zhvi_px', 'zhvi_idx', 'NbrLivingUnits', 'SqFtLot', 'SqFtTotLiving', 'SqFtFinBasement', 'Bathrooms', 
                'Bedrooms', 'BldgGrade',   'YrBuilt', 'YrRenovated', 'TrafficNoise', 'LandVal', 'ImpsVal', 'ZipCode', 'NewConstruction']

label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice
0,405100,0.93,2,9373,2400,0,3.00,6,7,1991,0,0,70000,229000,98002,False,280000
1,404400,0.93,1,20156,3764,1452,3.75,4,10,2005,0,0,203000,590000,98166,True,1000000
2,425600,0.98,1,26036,2060,900,1.75,4,8,1947,0,0,183000,275000,98166,False,745000
3,418400,0.96,1,8618,3200,1640,3.75,5,7,1966,0,0,104000,229000,98168,False,425000
4,351600,0.81,1,8620,1720,0,1.75,4,7,1948,0,0,104000,205000,98168,False,240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27058,332400,0.76,1,1161,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,False,374000
27059,339400,0.78,1,1005,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,True,374000
27060,325300,0.75,1,11170,1070,0,1.00,4,6,1971,0,0,92000,130000,98055,False,165000
27061,400600,0.92,1,6223,1345,0,2.00,3,7,1939,0,0,103000,212000,98166,False,315000


In [6]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 16)
y.shape =  (27063, 1)


## Step 4 : Run Regression

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import  LassoLarsIC
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# algo + data --> model
# algo =  LinearRegression ()
#algo = LassoLarsIC(criterion='aic', max_iter=500)
algo = RandomForestRegressor()

model = algo.fit(x, np.ravel(y))  # <--- training
model



RandomForestRegressor()

## Step 5 : Predict

In [8]:
## TODO Predict on 'x'
predictions = model.predict(x)
predictions

array([317555.69, 907216.5 , 639146.74, ..., 167193.5 , 320711.6 ,
       493244.25])

In [9]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a.sample(10)

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice,predicted_price
11165,321600,0.74,1,1005,1540,0,2.25,3,8,2008,0,1,128000,441000,-1,False,429950,"432,639.50"
10795,330600,0.76,1,8523,3170,0,2.50,4,9,2008,0,3,311000,415000,-1,True,575000,"598,241.33"
9041,312000,0.72,1,1094,1550,170,2.00,2,9,2011,0,0,114000,551000,98112,True,510000,"528,781.35"
16677,318000,0.73,1,12409,1970,750,1.75,4,7,1968,0,0,227000,176000,98028,False,277800,"296,768.50"
11702,425600,0.98,1,3864,1400,0,1.75,2,8,1981,0,0,65000,151000,98198,False,310000,"302,452.00"
4145,425600,0.98,1,6600,1370,0,1.00,3,7,1963,0,1,77000,171000,98168,False,299950,"302,491.48"
24241,411100,0.94,1,7751,2330,940,2.75,4,8,1979,0,0,96000,138000,98023,False,329950,"326,673.38"
4063,407400,0.94,1,13775,2340,490,2.00,3,8,1985,0,0,122000,258000,98038,False,430000,"430,113.90"
20891,411100,0.94,1,7575,1610,500,1.75,4,7,1965,0,0,91000,162000,98003,False,285000,"289,301.00"
13049,318000,0.73,1,7200,1660,0,2.25,4,7,1990,0,0,78000,145000,98003,False,180000,"184,291.50"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

**Q==> Also inspect R2 value.  Is it decent?**

In [10]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "feature_importance": model.feature_importances_})
coef.sort_values('feature_importance', ascending=False)

# will add upto 1.0 (100%)

,input_column,feature_importance
13,ImpsVal,0.57
12,LandVal,0.29
9,YrBuilt,0.03
4,SqFtTotLiving,0.02
0,zhvi_px,0.02
1,zhvi_idx,0.02
3,SqFtLot,0.02
8,BldgGrade,0.01
5,SqFtFinBasement,0.01
14,ZipCode,0.01


In [12]:
model.n_features_in_

16

In [11]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

## TODO : find R2 by comparing 'y' and 'predictions'
print ("R2 : " , r2_score(y, predictions))

print ("MSE : ", mean_squared_error(y, predictions))
print ("RMSE : ", sqrt(mean_squared_error(y, predictions)) )

R2 :  0.9832581606119447
MSE :  1967534633.668428
RMSE :  44356.900631902


## Discussion

With R2 approaching 1.0, it is very likely the RF algorithm is overfitting :-)

We need to use the hold-out method to split the data intro train/test sets to fix this.  More on this later.